In [ ]:
import os
import random
import cv2 as cv
import numpy as np
import tensorflow as tf
from sklearn import metrics
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [ ]:
# Upload train/test data

np.random.seed(7)
female = []
male = []

for i in os.listdir('C:/Users/72684/Desktop/Reinforcement Learning/Female'):
    path = 'C:/Users/72684/Desktop/Reinforcement Learning/Female/{}'.format(i)
    female.append([cv.imread(path, cv.IMREAD_COLOR) / 255, 0])

for i in os.listdir('C:/Users/72684/Desktop/Reinforcement Learning/Male'):
    path = 'C:/Users/72684/Desktop/Reinforcement Learning/Male/{}'.format(i)
    male.append([cv.imread(path, cv.IMREAD_COLOR) / 255, 1])

female = np.array(female)
male = np.array(male)

f_trn, f_tst = train_test_split(female, test_size = 0.2)
m_trn, m_tst = train_test_split(male, test_size = 0.2)

train = []
test = []

for i in range(len(f_trn)):
    train.append(f_trn[i])
    
for i in range(len(m_trn)):
    train.append(m_trn[i])

for i in range(len(f_tst)):
    test.append(f_tst[i])

for i in range(len(m_tst)):
    test.append(m_tst[i])

np.random.seed(7)
random.shuffle(train)
np.random.seed(7)
random.shuffle(test)

train = np.array(train)
test = np.array(test)

trn_d = []
tst_d = []

trn_l = np.zeros(len(train))
tst_l = np.zeros(len(test))

for i in range(len(train)):
    trn_l[i] = train[i, 1]
    trn_d.append(train[i, 0])

for i in range(len(test)):
    tst_l[i] = test[i, 1]
    tst_d.append(test[i, 0])

trn_d = np.array(trn_d)
tst_d = np.array(tst_d)

In [ ]:
# Define your model and its structural hyperparameters there

def build_model(krnl1, krnl2, krnl3, dns_lr, maps):
    tf.random.set_seed(7)
    
    mdl = Sequential()
    
    mdl.add(Conv2D(maps, krnl1, activation = 'relu', input_shape = (48, 48, 3), padding = "same"))
    mdl.add(MaxPooling2D(2, 2))
    
    mdl.add(Conv2D(2 * maps, krnl2, activation = 'relu', padding = "same"))
    mdl.add(MaxPooling2D(2, 2))
    
    mdl.add(Conv2D(4 * maps, krnl3, activation = 'relu', padding = "same"))
    
    mdl.add(Flatten())
    mdl.add(Dense(dns_lr, activation = 'relu'))
    mdl.add(Dense(1, activation = 'sigmoid'))
    
    mdl.compile(loss = 'binary_crossentropy', optimizer = 'adagrad', metrics = ['accuracy'])
    
    return mdl

## Greedy

In [ ]:
# Hyperparameters search algorithm

def hyp_prm_fit(epochs, btch_sz, krnl1, krnl2, krnl3, dns_lr, maps, trn_d, trn_l, tst_d, tst_l): # Define your system's
    h_p = np.array([epochs, btch_sz, dns_lr, maps, krnl1, krnl2, krnl3])                         # hyperparameters including
    steps_cntns = np.array([0.6, 0.8, 1, 1.2, 1.4])                                              # train hyperparameters
    steps_dscrt = np.array([3, 5, 7, 9], int) # steps_cntns steps for continuous values (not limited with some set of values)
    dt_tp = np.array(['int', 'int', 'int', 'int']) # steps_dscrt for discrete values (possible kernel sizes and so on)
    acc = 0 # dt_tp is to set continuous value's datatype for each hyperparameter
    cnt = 0
    acc_hst = [] # Change names and amount of variables below if you have another set of hyperparameters
    epchs_hst = []
    btch_sz_hst = []
    krnl1_hst = []
    krnl2_hst = []
    krnl3_hst = []
    dns_lr_hst = []
    maps_hst = []
    
    mdl = build_model((krnl1, krnl1), (krnl2, krnl2), (krnl3, krnl3), dns_lr, maps) # Set structural hyperparameters
    tf.random.set_seed(7) # Set train hyperparameters below
    mdl.fit(trn_d, trn_l, epochs = epochs, batch_size = btch_sz, validation_data = (tst_d, tst_l), verbose = False)
    acc = mdl.evaluate(tst_d, tst_l, verbose = False)[1]
    acc_hst.append(acc) # Change names and amount of variables below if you have another set of hyperparameters
    epchs_hst.append(epochs)
    btch_sz_hst.append(btch_sz)
    krnl1_hst.append(krnl1)
    krnl2_hst.append(krnl2)
    krnl3_hst.append(krnl3)
    dns_lr_hst.append(dns_lr)
    maps_hst.append(maps)
    print(acc)
    
    for episode in range(10): # Amount of episodes to find hyperparameters
        for i in range(len(h_p)):
            if i <= 3: # This value + 1 corresponds to amount of continuous hyperparameters
                h_p_fld = np.zeros(len(steps_cntns))
                h_p_fld = np.array(list(set(np.array(h_p[i] * steps_cntns, dt_tp[i]))))
                new_acc = np.zeros(len(h_p_fld))
                rsrv = h_p[i]
                print(h_p)
                print(h_p_fld)
            
            else: # There the work with discrete hyperparameters starts
                h_p_fld = np.zeros(len(steps_dscrt))
                h_p_fld = np.array(steps_dscrt)
                new_acc = np.zeros(len(h_p_fld))
                rsrv = h_p[i]
                print(h_p)
                print(h_p_fld)
            
            for val in range(len(h_p_fld)):
                h_p[i] = h_p_fld[val] # h_p stores current values of hyperparameters for training
                
                mdl = build_model((h_p[4], h_p[4]), (h_p[5], h_p[5]), (h_p[6], h_p[6]), h_p[2], h_p[3])
                tf.random.set_seed(7)
                mdl.fit(trn_d, trn_l, epochs = h_p[0], batch_size = h_p[1], validation_data = (tst_d, tst_l), verbose = False)
                new_acc[val] = mdl.evaluate(tst_d, tst_l, verbose = False)[1]
            
            
            if np.max(new_acc) > acc:
                cnt = 0
            
            else:
                cnt += 1
            
            if h_p_fld[np.argmax(new_acc)] == rsrv:
                res_acc = np.copy(new_acc)
                res_acc[np.argmax(new_acc)] = 0
                
                if np.max(res_acc) == np.max(new_acc):
                    h_p[i] = h_p_fld[np.argmax(res_acc)]
                    acc_hst.append(np.max(res_acc))
                    acc = np.max(res_acc)
                
                else:
                    h_p[i] = h_p_fld[np.argmax(new_acc)]
                    acc_hst.append(np.max(new_acc))
                    acc = np.max(new_acc)
            
            else:
                h_p[i] = h_p_fld[np.argmax(new_acc)]
                acc_hst.append(np.max(new_acc))
                acc = np.max(new_acc)
            
            print(new_acc)
            
            if i == 0: # Change names and amount of variables below if you have another set of hyperparameters
                epchs_hst.append(h_p[i])
            
            if i == 1:
                btch_sz_hst.append(h_p[i])
            
            if i == 2:
                dns_lr_hst.append(h_p[i])
            
            if i == 3:
                maps_hst.append(h_p[i])
            
            if i == 4:
                krnl1_hst.append(h_p[i])
            
            if i == 5:
                krnl2_hst.append(h_p[i])
            
            if i == 6:
                krnl3_hst.append(h_p[i])
            
            if cnt > 2 * len(h_p): # Search breaking criterion
                break
        
        if cnt > 2 * len(h_p):
            break
    
    acc_hst = np.array(acc_hst) # Change names and amount of variables below if you have another set of hyperparameters
    epchs_hst = np.array(epchs_hst)
    btch_sz_hst = np.array(btch_sz_hst)
    krnl1_hst = np.array(krnl1_hst)
    krnl2_hst = np.array(krnl2_hst)
    krnl3_hst = np.array(krnl3_hst)
    dns_lr_hst = np.array(dns_lr_hst)
    maps_hst = np.array(maps_hst)
    print(h_p)
    
    plt.plot(range(len(acc_hst)), acc_hst) # Change amount of plots below with respect to amount of hyperparameters (except acc)
    plt.grid()
    plt.xlabel('Hyperparameter changes')
    plt.ylabel('Accuracy')
    plt.show()
    
    plt.plot(range(len(epchs_hst)), epchs_hst)
    plt.grid()
    plt.xlabel('Hyperparameter epochs changes')
    plt.ylabel('Amount of epochs')
    plt.show()
    
    plt.plot(range(len(btch_sz_hst)), btch_sz_hst)
    plt.grid()
    plt.xlabel('Hyperparameter batch size changes')
    plt.ylabel('Batch size')
    plt.show()
    
    plt.plot(range(len(krnl1_hst)), krnl1_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 1 size changes')
    plt.ylabel('Kernel 1 size')
    plt.show()
    
    plt.plot(range(len(krnl2_hst)), krnl2_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 2 size changes')
    plt.ylabel('Kernel 2 size')
    plt.show()
    
    plt.plot(range(len(krnl3_hst)), krnl3_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 3 size changes')
    plt.ylabel('Kernel 3 size')
    plt.show()
    
    plt.plot(range(len(dns_lr_hst)), dns_lr_hst)
    plt.grid()
    plt.xlabel('Hyperparameter size of dense layer changes')
    plt.ylabel('Size of dense layer')
    plt.show()
    
    plt.plot(range(len(maps_hst)), maps_hst)
    plt.grid()
    plt.xlabel('Hyperparameter basic amount of maps changes')
    plt.ylabel('Basic amount of maps')
    plt.show()
    
    return np.array([acc_hst, epchs_hst, btch_sz_hst, krnl1_hst, krnl2_hst, krnl3_hst, dns_lr_hst, maps_hst, h_p])

In [ ]:
res_1 = hyp_prm_fit(15, 15, 3, 3, 3, 100, 16, trn_d, trn_l, tst_d, tst_l) # Set initial value of each hyperparameter

## e-greedy

In [ ]:
def hyp_prm_fit(epochs, btch_sz, krnl1, krnl2, krnl3, dns_lr, maps, trn_d, trn_l, tst_d, tst_l):
    h_p = np.array([epochs, btch_sz, dns_lr, maps, krnl1, krnl2, krnl3])
    steps_cntns = np.array([0.6, 0.8, 1, 1.2, 1.4])
    steps_dscrt = np.array([3, 5, 7, 9], int)
    dt_tp = np.array(['int', 'int', 'int', 'int'])
    acc = 0
    cnt = 0
    acc_hst = []
    epchs_hst = []
    btch_sz_hst = []
    krnl1_hst = []
    krnl2_hst = []
    krnl3_hst = []
    dns_lr_hst = []
    maps_hst = []
    
    mdl = build_model((krnl1, krnl1), (krnl2, krnl2), (krnl3, krnl3), dns_lr, maps)
    tf.random.set_seed(7)
    mdl.fit(trn_d, trn_l, epochs = epochs, batch_size = btch_sz, validation_data = (tst_d, tst_l), verbose = False)
    acc = mdl.evaluate(tst_d, tst_l, verbose = False)[1]
    acc_hst.append(acc)
    epchs_hst.append(epochs)
    btch_sz_hst.append(btch_sz)
    krnl1_hst.append(krnl1)
    krnl2_hst.append(krnl2)
    krnl3_hst.append(krnl3)
    dns_lr_hst.append(dns_lr)
    maps_hst.append(maps)
    print(acc)
    
    for episode in range(10):
        for i in range(len(h_p)):
            if i <= 3:
                h_p_fld = np.zeros(len(steps_cntns))
                h_p_fld = np.array(list(set(np.array(h_p[i] * steps_cntns, dt_tp[i]))))
                new_acc = np.zeros(len(h_p_fld))
                rsrv = h_p[i]
                print(h_p)
                print(h_p_fld)
            
            else:
                h_p_fld = np.zeros(len(steps_dscrt))
                h_p_fld = np.array(steps_dscrt)
                new_acc = np.zeros(len(h_p_fld))
                rsrv = h_p[i]
                print(h_p)
                print(h_p_fld)
            
            for val in range(len(h_p_fld)):
                h_p[i] = h_p_fld[val]
                
                mdl = build_model((h_p[4], h_p[4]), (h_p[5], h_p[5]), (h_p[6], h_p[6]), h_p[2], h_p[3])
                tf.random.set_seed(7)
                mdl.fit(trn_d, trn_l, epochs = h_p[0], batch_size = h_p[1], validation_data = (tst_d, tst_l), verbose = False)
                new_acc[val] = mdl.evaluate(tst_d, tst_l, verbose = False)[1]
            
            
            if np.random.rand() > 0.95:
                chc = np.random.choice(len(h_p_fld))
                
                if new_acc[chc] > acc:
                    cnt = 0
                
                else:
                    cnt += 1
                
                h_p[i] = h_p_fld[chc]
                acc_hst.append(new_acc[chc])
                acc = new_acc[chc]
            
            else:
                if np.max(new_acc) > acc:
                    cnt = 0
                
                else:
                    cnt += 1
                
                if h_p_fld[np.argmax(new_acc)] == rsrv:
                    res_acc = np.copy(new_acc)
                    res_acc[np.argmax(new_acc)] = 0
                    
                    if np.max(res_acc) == np.max(new_acc):
                        h_p[i] = h_p_fld[np.argmax(res_acc)]
                        acc_hst.append(np.max(res_acc))
                        acc = np.max(res_acc)
                    
                    else:
                        h_p[i] = h_p_fld[np.argmax(new_acc)]
                        acc_hst.append(np.max(new_acc))
                        acc = np.max(new_acc)
                
                else:
                    h_p[i] = h_p_fld[np.argmax(new_acc)]
                    acc_hst.append(np.max(new_acc))
                    acc = np.max(new_acc)
            
            print(new_acc)
            
            if i == 0:
                epchs_hst.append(h_p[i])
            
            if i == 1:
                btch_sz_hst.append(h_p[i])
            
            if i == 2:
                dns_lr_hst.append(h_p[i])
            
            if i == 3:
                maps_hst.append(h_p[i])
            
            if i == 4:
                krnl1_hst.append(h_p[i])
            
            if i == 5:
                krnl2_hst.append(h_p[i])
            
            if i == 6:
                krnl3_hst.append(h_p[i])
            
            if cnt > 2 * len(h_p):
                break
        
        if cnt > 2 * len(h_p):
            break
    
    acc_hst = np.array(acc_hst)
    epchs_hst = np.array(epchs_hst)
    btch_sz_hst = np.array(btch_sz_hst)
    krnl1_hst = np.array(krnl1_hst)
    krnl2_hst = np.array(krnl2_hst)
    krnl3_hst = np.array(krnl3_hst)
    dns_lr_hst = np.array(dns_lr_hst)
    maps_hst = np.array(maps_hst)
    print(h_p)
    
    plt.plot(range(len(acc_hst)), acc_hst)
    plt.grid()
    plt.xlabel('Hyperparameter changes')
    plt.ylabel('Accuracy')
    plt.show()
    
    plt.plot(range(len(epchs_hst)), epchs_hst)
    plt.grid()
    plt.xlabel('Hyperparameter epochs changes')
    plt.ylabel('Amount of epochs')
    plt.show()
    
    plt.plot(range(len(btch_sz_hst)), btch_sz_hst)
    plt.grid()
    plt.xlabel('Hyperparameter batch size changes')
    plt.ylabel('Batch size')
    plt.show()
    
    plt.plot(range(len(krnl1_hst)), krnl1_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 1 size changes')
    plt.ylabel('Kernel 1 size')
    plt.show()
    
    plt.plot(range(len(krnl2_hst)), krnl2_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 2 size changes')
    plt.ylabel('Kernel 2 size')
    plt.show()
    
    plt.plot(range(len(krnl3_hst)), krnl3_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 3 size changes')
    plt.ylabel('Kernel 3 size')
    plt.show()
    
    plt.plot(range(len(dns_lr_hst)), dns_lr_hst)
    plt.grid()
    plt.xlabel('Hyperparameter size of dense layer changes')
    plt.ylabel('Size of dense layer')
    plt.show()
    
    plt.plot(range(len(maps_hst)), maps_hst)
    plt.grid()
    plt.xlabel('Hyperparameter basic amount of maps changes')
    plt.ylabel('Basic amount of maps')
    plt.show()
    
    return np.array([acc_hst, epchs_hst, btch_sz_hst, krnl1_hst, krnl2_hst, krnl3_hst, dns_lr_hst, maps_hst, h_p])

In [ ]:
res_2 = hyp_prm_fit(15, 15, 3, 3, 3, 100, 16, trn_d, trn_l, tst_d, tst_l)

## Decaying e-greedy

In [ ]:
def hyp_prm_fit(epochs, btch_sz, krnl1, krnl2, krnl3, dns_lr, maps, trn_d, trn_l, tst_d, tst_l):
    h_p = np.array([epochs, btch_sz, dns_lr, maps, krnl1, krnl2, krnl3])
    steps_cntns = np.array([0.6, 0.8, 1, 1.2, 1.4])
    steps_dscrt = np.array([3, 5, 7, 9], int)
    dt_tp = np.array(['int', 'int', 'int', 'int'])
    acc = 0
    cnt = 0
    acc_hst = []
    epchs_hst = []
    btch_sz_hst = []
    krnl1_hst = []
    krnl2_hst = []
    krnl3_hst = []
    dns_lr_hst = []
    maps_hst = []
    
    mdl = build_model((krnl1, krnl1), (krnl2, krnl2), (krnl3, krnl3), dns_lr, maps)
    tf.random.set_seed(7)
    mdl.fit(trn_d, trn_l, epochs = epochs, batch_size = btch_sz, validation_data = (tst_d, tst_l), verbose = False)
    acc = mdl.evaluate(tst_d, tst_l, verbose = False)[1]
    acc_hst.append(acc)
    epchs_hst.append(epochs)
    btch_sz_hst.append(btch_sz)
    krnl1_hst.append(krnl1)
    krnl2_hst.append(krnl2)
    krnl3_hst.append(krnl3)
    dns_lr_hst.append(dns_lr)
    maps_hst.append(maps)
    print(acc)
    
    for episode in range(10):
        for i in range(len(h_p)):
            if i <= 3:
                h_p_fld = np.zeros(len(steps_cntns))
                h_p_fld = np.array(list(set(np.array(h_p[i] * steps_cntns, dt_tp[i]))))
                new_acc = np.zeros(len(h_p_fld))
                rsrv = h_p[i]
                print(h_p)
                print(h_p_fld)
            
            else:
                h_p_fld = np.zeros(len(steps_dscrt))
                h_p_fld = np.array(steps_dscrt)
                new_acc = np.zeros(len(h_p_fld))
                rsrv = h_p[i]
                print(h_p)
                print(h_p_fld)
            
            for val in range(len(h_p_fld)):
                h_p[i] = h_p_fld[val]
                
                mdl = build_model((h_p[4], h_p[4]), (h_p[5], h_p[5]), (h_p[6], h_p[6]), h_p[2], h_p[3])
                tf.random.set_seed(7)
                mdl.fit(trn_d, trn_l, epochs = h_p[0], batch_size = h_p[1], validation_data = (tst_d, tst_l), verbose = False)
                new_acc[val] = mdl.evaluate(tst_d, tst_l, verbose = False)[1]
            
            
            if np.random.rand() > (1 - 0.2 / (episode + 1)):
                chc = np.random.choice(len(h_p_fld))
                
                if new_acc[chc] > acc:
                    cnt = 0
                
                else:
                    cnt += 1
                
                h_p[i] = h_p_fld[chc]
                acc_hst.append(new_acc[chc])
                acc = new_acc[chc]
            
            else:
                if np.max(new_acc) > acc:
                    cnt = 0
                
                else:
                    cnt += 1
                
                if h_p_fld[np.argmax(new_acc)] == rsrv:
                    res_acc = np.copy(new_acc)
                    res_acc[np.argmax(new_acc)] = 0
                    
                    if np.max(res_acc) == np.max(new_acc):
                        h_p[i] = h_p_fld[np.argmax(res_acc)]
                        acc_hst.append(np.max(res_acc))
                        acc = np.max(res_acc)
                    
                    else:
                        h_p[i] = h_p_fld[np.argmax(new_acc)]
                        acc_hst.append(np.max(new_acc))
                        acc = np.max(new_acc)
                
                else:
                    h_p[i] = h_p_fld[np.argmax(new_acc)]
                    acc_hst.append(np.max(new_acc))
                    acc = np.max(new_acc)
            
            print(new_acc)
            
            if i == 0:
                epchs_hst.append(h_p[i])
            
            if i == 1:
                btch_sz_hst.append(h_p[i])
            
            if i == 2:
                dns_lr_hst.append(h_p[i])
            
            if i == 3:
                maps_hst.append(h_p[i])
            
            if i == 4:
                krnl1_hst.append(h_p[i])
            
            if i == 5:
                krnl2_hst.append(h_p[i])
            
            if i == 6:
                krnl3_hst.append(h_p[i])
            
            if cnt > 2 * len(h_p):
                break
        
        if cnt > 2 * len(h_p):
            break
    
    acc_hst = np.array(acc_hst)
    epchs_hst = np.array(epchs_hst)
    btch_sz_hst = np.array(btch_sz_hst)
    krnl1_hst = np.array(krnl1_hst)
    krnl2_hst = np.array(krnl2_hst)
    krnl3_hst = np.array(krnl3_hst)
    dns_lr_hst = np.array(dns_lr_hst)
    maps_hst = np.array(maps_hst)
    print(h_p)
    
    plt.plot(range(len(acc_hst)), acc_hst)
    plt.grid()
    plt.xlabel('Hyperparameter changes')
    plt.ylabel('Accuracy')
    plt.show()
    
    plt.plot(range(len(epchs_hst)), epchs_hst)
    plt.grid()
    plt.xlabel('Hyperparameter epochs changes')
    plt.ylabel('Amount of epochs')
    plt.show()
    
    plt.plot(range(len(btch_sz_hst)), btch_sz_hst)
    plt.grid()
    plt.xlabel('Hyperparameter batch size changes')
    plt.ylabel('Batch size')
    plt.show()
    
    plt.plot(range(len(krnl1_hst)), krnl1_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 1 size changes')
    plt.ylabel('Kernel 1 size')
    plt.show()
    
    plt.plot(range(len(krnl2_hst)), krnl2_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 2 size changes')
    plt.ylabel('Kernel 2 size')
    plt.show()
    
    plt.plot(range(len(krnl3_hst)), krnl3_hst)
    plt.grid()
    plt.xlabel('Hyperparameter kernel 3 size changes')
    plt.ylabel('Kernel 3 size')
    plt.show()
    
    plt.plot(range(len(dns_lr_hst)), dns_lr_hst)
    plt.grid()
    plt.xlabel('Hyperparameter size of dense layer changes')
    plt.ylabel('Size of dense layer')
    plt.show()
    
    plt.plot(range(len(maps_hst)), maps_hst)
    plt.grid()
    plt.xlabel('Hyperparameter basic amount of maps changes')
    plt.ylabel('Basic amount of maps')
    plt.show()
    
    return np.array([acc_hst, epchs_hst, btch_sz_hst, krnl1_hst, krnl2_hst, krnl3_hst, dns_lr_hst, maps_hst, h_p])

In [ ]:
res_3 = hyp_prm_fit(15, 15, 3, 3, 3, 100, 16, trn_d, trn_l, tst_d, tst_l)